### Importing Dependencies

In [ ]:
#uv add pandas numpy matplotlib seaborn pyarrow

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from dotenv import load_dotenv
import os
from IPython.display import display

### Assigning Constants

In [ ]:
load_dotenv()
URL = os.getenv("URL")

### Downloading the file

In [ ]:
def download_file(url):
     response = requests.get(url)
     if "content-disposition" in response.headers:
         content_disposition = response.headers["content-disposition"]
         filename = content_disposition.split("filename=")[1]
     else:
         filename = url.split("/")[-1]
     with open(filename, mode="wb") as file:
         file.write(response.content)
     print(f"Downloaded file {filename}")

In [ ]:
download_file(URL)

### Unzipping the downloaded file

In [ ]:
import gzip

def gunzip(source_filepath, dest_filepath, block_size=65536):
    with gzip.open(source_filepath, 'rb') as s_file, \
            open(dest_filepath, 'wb') as d_file:
        while True:
            block = s_file.read(block_size)
            if not block:
                break
            else:
                d_file.write(block)

gunzip("Tyroo-dummy-data.csv.gz","Tyroo-dummy-data.csv")

Optimizing the read speeds

In [3]:
df = pd.read_csv("Tyroo-dummy-data.csv", engine='pyarrow')

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
df.head()

,platform_commission_rate,venture_category3_name_en,product_small_img,deeplink,availability,image_url_5,number_of_reviews,is_free_shipping,promotion_price,venture_category2_name_en,current_price,product_medium_img,venture_category1_name_en,brand_name,image_url_4,description,seller_url,product_commission_rate,product_name,sku_id,seller_rating,bonus_commission_rate,business_type,business_area,image_url_2,discount_percentage,seller_name,product_url,product_id,venture_category_name_local,rating_avg_value,product_big_img,image_url_3,price
0,0.07,Parts & Spares,https://my-live.slatic.net/p/98b201dcb23a6f15c...,lazada://my/d?uri=https://www.lazada.com.my/pr...,in stock,None,20,0,16.74,Automotive,17.00,https://my-live.slatic.net/p/98b201dcb23a6f15c...,Motors,Perodua,None,None,https://www.lazada.com.my/shop/z-one-accessori...,0.07,Myvi Viva side bumper clip/Air Panel/wiper pan...,14676226228,99.0,0.0,Lazada,Marketplace,https://my-live.slatic.net/p/c14fc6fdbb240010a...,0.0,Z ONE ACCESSORIES WAREHOUSE SDN BHD,https://www.lazada.com.my/products/myvi-viva-s...,1352896491,Clips,5.0,https://my-live.slatic.net/p/98b201dcb23a6f15c...,None,17.00
1,0.1,Hair Care,https://my-live.slatic.net/p/90097ebc33ddd0641...,lazada://my/d?uri=https://www.lazada.com.my/pr...,in stock,https://my-live.slatic.net/p/90ffe96b5f53d38cf...,115,0,39.9,Personal Care,39.90,https://my-live.slatic.net/p/90097ebc33ddd0641...,Beauty,No Brand,https://my-live.slatic.net/p/4be2f46fee5858ca9...,KEY BENEFITSï¼Repairs damaged and compromised...,https://www.lazada.com.my/shop/olaplex-haircar...,0.22,（Buy one get one）Olaplex No.7 Bonding Oil 30ml...,22263928224,96.0,0.12,Lazada,Marketplace,https://my-live.slatic.net/p/8f699d3a2f66b0e45...,73.4,OLAPLEX HAIRCARE Store,https://www.lazada.com.my/products/buy-one-get...,3856537822,Hair Treatments,4.930434782608696,https://my-live.slatic.net/p/90097ebc33ddd0641...,https://my-live.slatic.net/p/140edb5b3b8d13009...,150.00
2,0.07,Electronics,https://my-live.slatic.net/p/35c1806bcb2b6895f...,lazada://my/d?uri=https://www.lazada.com.my/pr...,in stock,https://my-live.slatic.net/p/bd0e9b3b7a0120bdf...,185,0,609.0,Automotive,619.00,https://my-live.slatic.net/p/35c1806bcb2b6895f...,Motors,70mai,https://my-live.slatic.net/p/aefad376b4d670f33...,None,https://www.lazada.com.my/shop/70mai,0.11000000000000001,70mai S500 Rearview Dashcam Wide with Night Vi...,4335984780,99.0,0.04,LazMall,Marketplace,https://my-live.slatic.net/p/76442e587a6b97ecf...,15.09,70mai,https://www.lazada.com.my/products/70mai-s500-...,1353246653,On-Dash Video,4.951351351351351,https://my-live.slatic.net/p/35c1806bcb2b6895f...,https://my-live.slatic.net/p/b397560f5f47697dd...,729.00
3,0.07,Disposable Napkins,https://my-live.slatic.net/p/06e24c1ead0e47cb3...,lazada://my/d?uri=https://www.lazada.com.my/pr...,in stock,https://my-live.slatic.net/p/3b635d52337db761f...,33496,0,0.0,Disposables,1.06,https://my-live.slatic.net/p/06e24c1ead0e47cb3...,Kitchen & Dining,No Brand,https://my-live.slatic.net/p/b8432e0ce5b55933e...,"Size:50pcs/Roll, The patterns are randomized.T...",https://www.lazada.com.my/shop/oiga,0.12000000000000001,tissues paper Disposable Cleaning Cloths Wipe ...,21472602354,98.0,0.05,LazGlobal,Crossborder,https://my-live.slatic.net/p/67e9321adc531ab01...,74.94,OIGA,https://www.lazada.com.my/products/tissues-pap...,3772733731,Disposable Napkins,4.94632194888942,https://my-live.slatic.net/p/06e24c1ead0e47cb3...,https://my-live.slatic.net/p/c03b8685d5e09ed9c...,4.23
4,0.1,Serum & Essence,https://my-live.slatic.net/p/e17a59e38fc2418f8...,lazada://my/d?uri=https://www.lazada.com.my/pr...,in stock,https://my-live.slatic.net/p/0c7fe68f5a8a9826d...,20378,0,1.2,Skin Care,1.20,https://my-live.slatic.net/p/e17a59e38fc2418f8...,Beauty,No Brand,https://my-live.slatic.net/p/17c4711cec6168958...,None,https://www.lazada.com.my/shop/youbuy-official,0.15000000000000002,Natural aloe vera gel 300g containing plant es...,21755692333,98.0,0.05,LazGlobal,Crossborder,https://my-live.slatic.ne

### Preliminary Analysis

In [61]:
df.shape

(1000003, 34)

In [ ]:
df.info()

We can see all columns are of Object type.



In [ ]:
df.describe(include="all")

Checking Null columns

In [31]:
def print_null_summary(df):
    null_counts = df.isnull().sum()
    null_percent = df.isnull().mean() * 100
    null_summary = pd.concat([null_counts, null_percent], axis=1, keys=["Nulls", "%"])
    print(null_summary[null_summary["Nulls"] > 0].sort_values(ascending=False, by="Nulls"))

In [63]:
print_null_summary(df)

                            Nulls          %
description                409226  40.922477
image_url_5                176940  17.693947
image_url_4                118370  11.836964
rating_avg_value            70596   7.059579
number_of_reviews           70110   7.010979
image_url_3                 61397   6.139682
image_url_2                 38834   3.883388
venture_category3_name_en   30458   3.045791
brand_name                   1761   0.176099
seller_rating                1726   0.172599
venture_category2_name_en      10   0.001000


These need to be handled.

Checking Cardinality

In [64]:
cardinality = df.nunique().sort_values(ascending=False)
print(cardinality)

sku_id                         1000001
deeplink                       1000001
product_url                    1000001
product_small_img               879077
product_medium_img              879077
product_big_img                 879076
product_id                      721266
product_name                    700570
image_url_2                     661124
image_url_3                     636076
image_url_4                     589277
image_url_5                     541001
description                     375049
seller_name                      48955
seller_url                       48955
price                            33266
current_price                    33248
rating_avg_value                 31773
promotion_price                  30299
brand_name                       16586
discount_percentage               9731
number_of_reviews                 4259
venture_category_name_local       3021
venture_category3_name_en         1266
venture_category2_name_en          250
product_commission_rate  

Some questions arise due to these ...

Some columns have number of unique values that are less than length of total rows.

This means they have duplicate values.

Duplicates are valid for some columns like, numerical columns of price etc,

and some textual columns of seller name, seller url, brand name, category name, business type, business area

and some boolean columns like availability and is free shipping.

But for some columns like product name, product id, description, we need to check if we have duplicates/how are they unique.

These columns have highest cardinality-

sku_id

deeplink

product_url

Summary -

Rows - 1,000,003

Columns - 34

Data Issues -

 - High Nulls

 - All columns are object

High cardinality columns

### Data Cleaning

Checking if column names are a part of cells

In [12]:
def print_bad_rows(df):
    col_names = set(df.columns.astype(str)) #get column names as strings

    mask = df.astype(str).isin(col_names) #convert all values to string (temporary) and check if they match any column name

    bad_row_indices = mask.any(axis=1) #identify rows with any such match

    bad_rows = df[bad_row_indices] #extract and count bad rows
    print(f"Found {bad_rows.shape[0]} suspicious rows.")
    display(bad_rows)
    return bad_row_indices

In [13]:
bad_row_indices = print_bad_rows(df)

Found 3 suspicious rows.


,platform_commission_rate,venture_category3_name_en,product_small_img,deeplink,availability,image_url_5,number_of_reviews,is_free_shipping,promotion_price,venture_category2_name_en,current_price,product_medium_img,venture_category1_name_en,brand_name,image_url_4,description,seller_url,product_commission_rate,product_name,sku_id,seller_rating,bonus_commission_rate,business_type,business_area,image_url_2,discount_percentage,seller_name,product_url,product_id,venture_category_name_local,rating_avg_value,product_big_img,image_url_3,price
5000,platform_commission_rate,venture_category3_name_en,product_small_img,deeplink,availability,image_url_5,number_of_reviews,is_free_shipping,promotion_price,venture_category2_name_en,current_price,product_medium_img,venture_category1_name_en,brand_name,image_url_4,description,seller_url,product_commission_rate,product_name,sku_id,seller_rating,bonus_commission_rate,business_type,business_area,image_url_2,discount_percentage,seller_name,product_url,product_id,venture_category_name_local,rating_avg_value,product_big_img,image_url_3,price
10001,platform_commission_rate,venture_category3_name_en,product_small_img,deeplink,availability,image_url_5,number_of_reviews,is_free_shipping,promotion_price,venture_category2_name_en,current_price,product_medium_img,venture_category1_name_en,brand_name,image_url_4,description,seller_url,product_commission_rate,product_name,sku_id,seller_rating,bonus_commission_rate,business_type,business_area,image_url_2,discount_percentage,seller_name,product_url,product_id,venture_category_name_local,rating_avg_value,product_big_img,image_url_3,price
15002,platform_commission_rate,venture_category3_name_en,product_small_img,deeplink,availability,image_url_5,number_of_reviews,is_free_shipping,promotion_price,venture_category2_name_en,current_price,product_medium_img,venture_category1_name_en,brand_name,image_url_4,description,seller_url,product_commission_rate,product_name,sku_id,seller_rating,bonus_commission_rate,business_type,business_area,image_url_2,discount_percentage,seller_name,product_url,product_id,venture_category_name_local,rating_avg_value,product_big_img,image_url_3,price


Drop these bad rows

In [14]:
df = df[~bad_row_indices].reset_index(drop=True)

### Handling Null Values

In [15]:
df.sample()

,platform_commission_rate,venture_category3_name_en,product_small_img,deeplink,availability,image_url_5,number_of_reviews,is_free_shipping,promotion_price,venture_category2_name_en,current_price,product_medium_img,venture_category1_name_en,brand_name,image_url_4,description,seller_url,product_commission_rate,product_name,sku_id,seller_rating,bonus_commission_rate,business_type,business_area,image_url_2,discount_percentage,seller_name,product_url,product_id,venture_category_name_local,rating_avg_value,product_big_img,image_url_3,price
966357,0.07,Car Care,https://my-live.slatic.net/p/4aaa09306b883fe5d...,lazada://my/d?uri=https://www.lazada.com.my/pr...,in stock,https://my-live.slatic.net/p/8caafec723a6ee296...,25,0,0.0,Automotive,18.00,https://my-live.slatic.net/p/4aaa09306b883fe5d...,Motors,No Brand,https://my-live.slatic.net/p/a7d643eb59ec02e18...,ð We are Registered SSM Company in Malaysia...,https://www.lazada.com.my/shop/mcartoc,0.07,Proton X70 Navigation Tempered Glass Screen Pr...,19079738189,98.0,0.0,Lazada,Marketplace,https://my-live.slatic.net/p/7a13f6c74b6635d36...,35.71,Mcartoc,https://www.lazada.com.my/products/proton-x70-...,3484624793,Protectants,4.96,https://my-live.slatic.net/p/4aaa09306b883fe5d...,https://my-live.slatic.net/p/4226bbde770fc58f8...,28.00


Numeric cols are converted to 0.0 numeric for NA

In [16]:
num_cols=[
    'platform_commission_rate', 'product_commission_rate', 
    'bonus_commission_rate', 'promotion_price', 'current_price', 
    'price', 'discount_percentage', 'number_of_reviews', 
    'rating_avg_value', 'seller_rating'
]

In [17]:
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0.0)

Let's check unique values in boolean columns

In [19]:
print(df['is_free_shipping'].unique())

['0']


In [20]:
print(df['availability'].unique())

['in stock']


Now we set is_free_shipping to False if its 0 and availability to False for out of stock and True for in stock

In [21]:
df['is_free_shipping'] = df['is_free_shipping'].fillna('0').astype(bool)
df['availability'] = df['availability'].fillna('out of stock')
df['availability'] = df['availability'].map({'in stock': True, 'out of stock': False})

In [22]:
print(df['availability'].unique())

[ True]


In [23]:
print(df['is_free_shipping'].unique())

[ True]


Now check for image_cols

In [26]:
df.sample()

,platform_commission_rate,venture_category3_name_en,product_small_img,deeplink,availability,image_url_5,number_of_reviews,is_free_shipping,promotion_price,venture_category2_name_en,current_price,product_medium_img,venture_category1_name_en,brand_name,image_url_4,description,seller_url,product_commission_rate,product_name,sku_id,seller_rating,bonus_commission_rate,business_type,business_area,image_url_2,discount_percentage,seller_name,product_url,product_id,venture_category_name_local,rating_avg_value,product_big_img,image_url_3,price
631879,0.07,Desk Organisers,https://my-live.slatic.net/p/b9bf5237423491ab5...,lazada://my/d?uri=https://www.lazada.com.my/pr...,True,https://my-live.slatic.net/p/f4e4ced7da4379039...,51.0,True,4.82,School & Office Equipment,4.88,https://my-live.slatic.net/p/b9bf5237423491ab5...,"Stationery, Craft & Gift Cards",Sokano,https://my-live.slatic.net/p/553e42eb200501c76...,None,https://www.lazada.com.my/shop/sokano-shop,0.13,SOKANO DZOFFICE 1028 DIY WoodenTable and Penci...,1199886437,92.0,0.06,Lazada,Marketplace,https://my-live.slatic.net/p/f8dbeb7133c41fac9...,62.17,Sokano Shop,https://www.lazada.com.my/products/sokano-dzof...,593434410,Desk Organisers,4.784314,https://my-live.slatic.net/p/b9bf5237423491ab5...,https://my-live.slatic.net/p/2a9600c4eb4c0c827...,12.9


Gather all cols with img in name and set missing values as empty string

In [27]:
image_cols = [col for col in df.columns if 'img' in col or 'image_url' in col]
df[image_cols] = df[image_cols].fillna('')

Same for cols with URLS

In [28]:
df['deeplink'] = df['deeplink'].fillna('')
df['product_url'] = df['product_url'].fillna('')
df['seller_url'] = df['seller_url'].fillna('')

Other text cols that denote some name or category will be set to Unknown

In [29]:
text_cols = [
    'venture_category3_name_en', 'venture_category2_name_en', 'venture_category1_name_en',
    'venture_category_name_local', 'brand_name', 'business_type', 'business_area',
    'product_name', 'seller_name'
]

df[text_cols] = df[text_cols].fillna('Unknown')

Checking for nulls again

In [32]:
print_null_summary(df)

              Nulls        %
description  409226  40.9226


Checking for nulls in description in different categories

In [33]:
df[df['description'].isna()]['venture_category1_name_en'].value_counts()

venture_category1_name_en
Women's Shoes and Clothing                64908
Beauty                                    38429
Groceries                                 25552
Bags and Travel                           23903
Fashion Accessories                       22265
Kitchen & Dining                          19663
Electronics Accessories                   16952
Motors                                    16570
Tools & Home Improvement                  15951
Small Appliances                          13435
Furniture & Organization                  12968
Toys & Games                              12614
Stationery, Craft & Gift Cards            11849
Lighting & Décor                          11661
Sports & Outdoors                         10188
Men's Clothing                             8816
Mother & Baby                              8807
Pet Supplies                               8694
Kids' Fashion                              8437
Health                                     6895
Bedding & Bath

As description null values are spread across different categories hence we cant drop them,

we can use some meaningful data in its place

We can put product name in place of missing description values

In [34]:
df.sample()

,platform_commission_rate,venture_category3_name_en,product_small_img,deeplink,availability,image_url_5,number_of_reviews,is_free_shipping,promotion_price,venture_category2_name_en,current_price,product_medium_img,venture_category1_name_en,brand_name,image_url_4,description,seller_url,product_commission_rate,product_name,sku_id,seller_rating,bonus_commission_rate,business_type,business_area,image_url_2,discount_percentage,seller_name,product_url,product_id,venture_category_name_local,rating_avg_value,product_big_img,image_url_3,price
16540,0.07,Rice,https://my-live.slatic.net/p/6db841975b4763cd7...,lazada://my/d?uri=https://www.lazada.com.my/pr...,True,https://my-live.slatic.net/p/263e93fc450992539...,1002.0,True,7.92,Food Staples & Cooking Essentials,8.0,https://my-live.slatic.net/p/6db841975b4763cd7...,Groceries,NOBRAND,https://my-live.slatic.net/p/d2f675ef1cc6843dd...,Natural Ten Whole Grains å¤©ç¶åè°·ç±³ 500g ...,https://www.lazada.com.my/shop/cheng-woh-medic...,0.11,天然十谷米 500g Natural Ten Whole Grains 营养早餐粥十谷杂粮组...,649888190,98.0,0.04,Lazada,Marketplace,https://my-live.slatic.net/p/a250b6be49948bff7...,0.0,Cheng Woh Medical Hall,https://www.lazada.com.my/products/500g-natura...,440673377,Mixed Rice,4.916168,https://my-live.slatic.net/p/6db841975b4763cd7...,https://my-live.slatic.net/p/aae3fc63049df3e6b...,8.0


In [35]:
df['description'] = df['description'].fillna(df['product_name'])

In [36]:
print_null_summary(df)

Empty DataFrame
Columns: [Nulls, %]
Index: []


Checking Description Column values

In [67]:
df['description'].value_counts().sort_values(ascending=False).reset_index().head(30)

,description,count
0,&nbsp;,2082
1,----------,1066
2,welcome to my shop!1. The items that can be or...,966
3,.,589
4,&nbsp;&nbsp;,402
5,"â¤Hello dear, welcome to visit~~~ â¤The good...",384
6,"Typical user:Online searches,Online maps,Messa...",374
7,welcome to my shop! 1. The items that can be o...,311
8,"Promotion: Free gifts, free shipping and store...",305
9,"Dear friend, thank you for shopping on my stor...",299


In [68]:
df['product_name'].value_counts().sort_values(ascending=False).reset_index().head(30)

,product_name,count
0,Cold and Relaxed Feeling Wear Water Soluble La...,152
1,Horien Eye Secret 38% Daily Disposable Hydroge...,80
2,Baju Tidur Budak Lelaki Perempuan Pyjamas Kids...,78
3,3D Carpet High Quality Lipat Packing Carpet Ve...,78
4,[Power 0.00~-6.00] Magister Contact Lens 1Pair...,73
5,SPRINGO Premium Soft Aloe Cotton Cute Cartoon ...,73
6,SMC Ready Stock🇲🇾Seluar Celana Panjang Lelaki ...,72
7,现货✨穿戴美甲24枚 Fake Nails 24PCS 穿戴甲假指甲片拆卸指甲贴穿戴式美甲贴...,72
8,20/24 Inch Abs Hardshell Lightweight Carry On ...,63
9,(10 pcs) Horien Eye Secret 38% Daily Disposabl...,62
